In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from scipy import stats

ImportError: cannot import name 'mean_absolute_percentage_error' from 'sklearn.metrics' (C:\Users\shav.vimelindiran\Anaconda3\envs\ml\lib\site-packages\sklearn\metrics\__init__.py)

In [2]:
# Load data
TFT_encoding_predictions = np.load('../data/model_preds/TFT_encoded_predictions.npy', allow_pickle=True)
TFT_predictions = np.load('../data/model_preds/TFT_predictions.npy', allow_pickle=True)
LSTM_predictions = np.load('../data/model_preds/LSTM_predictions.npy', allow_pickle=True)
ARIMA_predictions = np.load('../data/model_preds/ARIMA_predictions.npy', allow_pickle=True)
naive_predictions = np.load('../data/model_preds/naive_predictions.npy', allow_pickle=True)
tests = np.load('../data/model_preds/actual_sales.npy', allow_pickle=True)
data = pd.read_csv('../data/data_processed_plant_item_10_03_22.csv')

In [3]:
#classifications = np.load('forecast_classification.npy')

In [4]:
def create_ensemble_preds(clf, TFT_preds_clf):
    TFT_ensemble_predictions = np.zeros((len(clf), 6))
    for i in range(len(clf)):
        if clf[i] == 0:
            TFT_ensemble_predictions[i, :] = np.zeros((6))
        else:
            TFT_ensemble_predictions[i, :] = TFT_preds_clf[i, :]
    return TFT_ensemble_predictions
#TFT_ensemble_predictions =create_ensemble_preds(classifications, TFT_predictions)

In [5]:
def WMAPE_calc(preds, tests):
    flat_pred = np.array(preds).flatten()
    flat_test = np.array(tests).flatten()
    residuals = []
    for i in range(len(flat_pred)):
        residuals.append(abs(flat_test[i] - flat_pred[i]))
    WMAPE = 100 * (np.sum(residuals) / np.sum(flat_test))
    return WMAPE

def WMAPE2_calc(preds, tests):
    preds = preds + 1
    tests = tests + 1
    total_vol = np.sum(tests)
    residuals = 0
    total_vol = np.sum(tests)
    for i in range(preds.shape[0]):
        pred = preds[i, :]
        for t in range(len(pred)):
            residuals += abs(pred[t] - tests[i, t])
    WMAPE_2 = (np.sum(residuals) / total_vol) * 100
    return WMAPE_2

def metric_test(preds, tests):
    MSEs = []
    MAPEs = []
    MAE = []
    for i in range(preds.shape[0]):
        pred = preds[i, :]
        test = tests[i, :]
        MSEs.append(mean_squared_error(test, pred))
        MAPEs.append(mean_absolute_percentage_error(test + 1, pred + 1))
        MAE.append(mean_absolute_error(test, pred))
    WMAPE = WMAPE_calc(preds, tests)
    WMAPE_2 = WMAPE2_calc(preds, tests)
    #WMAPE_2 = WMAPE_calc_2(preds, tests)
    print('mean squared error:                              ', round(np.mean(MSEs), 2))
    print('mean absolute error:                             ', round(np.mean(MAE), 2))
    print('mean absolute percentage error:          ', round(np.mean(MAPEs), 2), '%')
    #print('mean absolute percentage error weighted: ', round(WMAPE, 2), '%')
    print('mean absolute percentage error weighted: ', round(WMAPE_2, 2), '%')

def compare_performance(model_dict, tests):
    for model in list(model_dict.keys()):
        print(model)
        print('')
        metric_test(model_dict[model], tests)
        print('')


model_dict = {'TFT encodings':TFT_encoding_predictions, 'TFT':TFT_predictions, 'LSTM':LSTM_predictions, 'ARIMA':ARIMA_predictions, 'naive':naive_predictions}
compare_performance(model_dict, tests)

TFT encodings

mean squared error:                               30747.87
mean absolute error:                              35.11
mean absolute percentage error:           4.64 %
mean absolute percentage error weighted:  71.48 %

TFT

mean squared error:                               28831.01
mean absolute error:                              34.45
mean absolute percentage error:           4.54 %
mean absolute percentage error weighted:  70.13 %

LSTM

mean squared error:                               151461.38
mean absolute error:                              79.53
mean absolute percentage error:           22.0 %
mean absolute percentage error weighted:  161.91 %

ARIMA

mean squared error:                               40993.61
mean absolute error:                              38.72
mean absolute percentage error:           4.47 %
mean absolute percentage error weighted:  78.82 %

naive

mean squared error:                               41915.37
mean absolute error:                   

In [6]:
print(tests[0:4])

[[203.   0.   0.   0.   0.   0.]
 [  0.   0.  10.   3.   0.   0.]
 [  0.   0.   8.   8.   4.   1.]
 [  0.  40.  38.   0.  78.  42.]]


In [7]:
def sum_over_plants(array, data):
    timeseries_list = list(data['timeseries'].unique())
    item_dict = {}
    for i in range(len(timeseries_list)):
        timeseries = data[data['timeseries'] == timeseries_list[i]]
        itemID = list(timeseries['ParentItemID'].unique())[0]
        if itemID not in list(item_dict.keys()):
            item_dict[itemID] = array[i, :]
        else:
            prev_vol = item_dict[itemID]
            #print(prev_vol)
            #print(array[i, :])
            new_vol = prev_vol + array[i, :]
            #print(new_vol)
            #print('')
            item_dict[itemID] = new_vol
    print(len(list(item_dict.keys())))
    print(len(data['ParentItemID'].unique()))
    print('')
    return item_dict

In [8]:
test_item_dict = sum_over_plants(tests, data)
ARIMA_preds_dict = sum_over_plants(ARIMA_predictions, data)
TFT_encodings_preds_dict = sum_over_plants(TFT_encoding_predictions, data)
TFT_preds_dict = sum_over_plants(TFT_predictions, data)
LSTM_preds_dict = sum_over_plants(LSTM_predictions, data)
naive_preds_dict = sum_over_plants(naive_predictions, data)

1068
1068

1068
1068

1068
1068

1068
1068

1068
1068

1068
1068



In [9]:
def WMAPE2_calc(pred_dict, test_dict, model):
    residuals = 0
    total_vol = np.sum(list(test_dict.values()))
    for itemID in list(pred_dict.keys()):
        pred = pred_dict[itemID] + 0.1
        test = test_dict[itemID] + 0.1
        for t in range(len(pred)):
            residuals += abs(pred[t] - test[t])
    WMAPE_2 = (np.sum(residuals) / total_vol) * 100
    print('Weighted and aggregated MAPE for:', model, ':', round(WMAPE_2, 2), '%')

In [10]:
models = ['TFT encodings', 'TFT', 'LSTM', 'ARIMA', 'Naive']
WMAPE2_calc(TFT_encodings_preds_dict, test_item_dict, models[0])
WMAPE2_calc(TFT_preds_dict, test_item_dict, models[1])
WMAPE2_calc(LSTM_preds_dict, test_item_dict, models[2])
WMAPE2_calc(ARIMA_preds_dict, test_item_dict, models[3])
WMAPE2_calc(naive_preds_dict, test_item_dict, models[4])

Weighted and aggregated MAPE for: TFT encodings : 57.05 %
Weighted and aggregated MAPE for: TFT : 54.63 %
Weighted and aggregated MAPE for: LSTM : 131.94 %
Weighted and aggregated MAPE for: ARIMA : 60.13 %
Weighted and aggregated MAPE for: Naive : 62.37 %
